# 학교 안전사고 현황 분석

- 코드 작성자 : 정하윤
- 코드 작성일 : 2024-07-06

맡은 부분 알아서 잘 정리해주세요!!🥰

## 0. 라이브러리 및 데이터 임포트

In [1]:
import pandas as pd
import numpy as np

In [4]:
df_2019 = pd.read_excel('../data/★2019~2023 학교안전사고 데이터_수정.xlsx',sheet_name='2019')
df_2020 = pd.read_excel('../data/★2019~2023 학교안전사고 데이터_수정.xlsx',sheet_name='2020')
df_2021 = pd.read_excel('../data/★2019~2023 학교안전사고 데이터_수정.xlsx',sheet_name='2021')
df_2022 = pd.read_excel('../data/★2019~2023 학교안전사고 데이터_수정.xlsx',sheet_name='2022')
df_2023 = pd.read_excel('../data/★2019~2023 학교안전사고 데이터_수정.xlsx',sheet_name='2023')

## 1. 학교 안전사고 현황 EDA

### 1-1. 연도별 사고 현황

### 1-2. 지역별 사고 현황

- 시도별 사고 현황
    - 시도별 총 사고건수
    - 시도별 연도별 전체 사고건수
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 학교 비율
        -  전체 학교 중 안전사고가 발생한 학교의 비율(2021년 동작구의 전체 학교 N곳 중 M곳의 학교에서 안전사고가 발생했다.)
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 건수
- 시군구별 사고 현황
    - 시군구별 총 사고 건수 (시도별에서 시도를 선택하면 확인할 수 있도록)
    - 시군구별 연도별 총 사고건수
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 학교 비율
        -  전체 학교 중 안전사고가 발생한 학교의 비율(2021년 동작구의 전체 학교 N곳 중 M곳의 학교에서 안전사고가 발생했다.)
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 건수

In [6]:
df_2023.교육청.unique()

'''
교육청(시도)
교육지원청(시군구) --> 시군구별 분석 가능
    교육지원청은 시·도 교육청의 하급 교육행정기관으로, 교육장(3급 또는 4급 상당)의 감독 아래 시·도의 교육·학예에 관한 사무를 일부 분장한다. 
'''

array(['제주시교육지원청', '거창교육지원청', '창원교육지원청', '통영교육지원청', '양산교육지원청', '경상남도교육청',
       '거제교육지원청', '김해교육지원청', '진주교육지원청', '함안교육지원청', '사천교육지원청', '밀양교육지원청',
       '영천교육지원청', '경상북도교육청', '김천교육지원청', '구미교육지원청', '경산교육지원청', '경주교육지원청',
       '포항교육지원청', '성주교육지원청', '익산교육지원청', '전라북도교육청', '전주교육지원청', '김제교육지원청',
       '천안교육지원청', '아산교육지원청', '논산교육지원청', '서천교육지원청', '공주교육지원청', '홍성교육지원청',
       '서산교육지원청', '부여교육지원청', '당진교육지원청', '보은교육지원청', '청주교육지원청', '충청북도교육청',
       '진천교육지원청', '영동교육지원청', '옥천교육지원청', '음성교육지원청', '군포의왕교육지원청', '부천교육지원청',
       '평택교육지원청', '시흥교육지원청', '의정부교육지원청', '수원교육지원청', '고양교육지원청',
       '동두천양주교육지원청', '파주교육지원청', '구리남양주교육지원청', '안산교육지원청', '안양과천교육지원청',
       '화성오산교육지원청', '광명교육지원청', '김포교육지원청', '포천교육지원청', '용인교육지원청', '안성교육지원청',
       '가평교육지원청', '광주하남교육지원청', '세종특별자치시교육청', '강남교육지원청', '울산교육청',
       '강북교육지원청', '서부교육지원청', '동부교육지원청', '남부교육지원청', '북부교육지원청', '인천광역시교육청',
       '해운대교육지원청', '동래교육지원청', '부산광역시교육청', '강동송파교육지원청', '강서양천교육지원청',
       '성동광진교육지원청', '서울특별시교육청', '동작관악교육지원청', '서귀포시교육지원청', '고성교육지원청',
  

### 1-3. 학교급별 사고 현황

- 학교급별 (연도별) 총 사고 수
- 학교급별 (연도별) 학년별 사고 수
- 학교급별 (연도별) 성별별 사고 수
- 학교급별 하루 평균 발생 사고 수 : 초등학교에선 하루에 평균 몇 건의 사고가 발생하는지?
- 학교급별 사고 내용 (사고 형태, 사과 당시 활동, 사고매개물)
    - 학교급별 사고 내용에 대한 비교 (초등학교는 어떤 사고가 주로 발생하고, 중학교는 어떤 사고가 주로 발생하는지를 파악하고자 하는 내용)

In [8]:
df_2023.학교급.unique()

array(['초등학교', '중학교', '고등학교', '유치원', '기타학교', '특수학교'], dtype=object)

In [35]:
df_2023.groupby('학교급').count()['구분']

학교급
고등학교    39133
기타학교      626
유치원      9861
중학교     71223
초등학교    71608
특수학교      726
Name: 구분, dtype: int64

In [25]:
'''
해당연도 사고 접수일 기준 데이터이므로 
사고 발생연도는 해당연도 외 다른 연도 데이터도 포함 ( 발생연도 기준 분석 필요 )
'''
temp = df_2023[df_2023['학교급']=='초등학교'].groupby(['사고발생일']).count()['구분']
temp


사고발생일
2004-03-19      1
2019-12-13      1
2020-11-10      1
2020-11-19      1
2020-12-07      1
             ... 
2023-12-23      1
2023-12-26    200
2023-12-27    163
2023-12-28     97
2023-12-29     28
Name: 구분, Length: 531, dtype: int64

In [31]:
'''
하루 평균 134건 실화야?? 이상치 제거 해야할듯 하루에 너무 많이 발생한 날은 제외시키고 다시보기...
'''
temp.describe()

count    531.000000
mean     134.854991
std      173.027579
min        1.000000
25%        2.000000
50%       13.000000
75%      342.500000
max      571.000000
Name: 구분, dtype: float64

### 1-4. 사고 내용 분석

- 장소별 사고 현황
    - 장소별 사고 발생 건수 (누적? 연도별?)
    - 장소별 하루 평균 사고 발생 건수 : 해당 장소에서 하루에 얼마나 사고가 발생하는지?
    - 장소별 사고 부위, 사고 형태
- 시간별 사고 현황
    - 계절별 사고 발생 현황 : 계절별로 어떤 사고가 많이 발생하는지 (계절별 예방 대책 제시 가능)
        - 계절별 사고 건수
        - 계절별 사고 부위
        - 계절별 사고당시활동
    - 사고 시간별 사고 현황 (수업시간, 쉬는시간, 식사시간 등)
        - 사고 시간별 사고 건수
            - 사고 발생 시각별 건수(오전, 점심, 오후 정도의 비교가 의미 있지 않을지? <br>
            : 사고 시간별 건수 분석은 그냥 학교에서의 전체 시간이 수업시간, 쉬는 시간으로 뭉뚱그려진 게 좀 아쉬워서 오전 오후의 비교도 하면 어떨지 생각해봄, 애들이 밥먹고 하교 시간이 다가오면 좀 신나서 더 위험하다든가.. 그런 추측)
        - 사고 시간별 하루 평균 사고 발생 건수 : 해당 시간 대에 하루에 얼마나 사고가 발생하는지?
        - 사고 시간별 사고 형태, 사고 당시 활동, 사고 매개물 <br>
        : 사고 시간별로 어떤 사고가 많이 발생하는지? 등하교 시간에 자주 발생하는 사고 내용, 점심시간에 자주 발생하는 사고 내용 등 파악


In [11]:
df_2023.사고장소.unique()

array(['운동장', '통로', '부속시설', '교실', '교외'], dtype=object)

In [12]:
df_2023.사고시간.unique()

array(['식사시간', '체육활동', '수업시간', '학교행사', '특별활동', '쉬는시간', '등하교', '기타'],
      dtype=object)

In [14]:
df_2023.사고발생시각.unique()

array(['12:40', '11:20', '11:50', '13:20', '10:00', '13:45', '14:30',
       '11:00', '18:00', '13:50', '13:10', '15:00', '11:25', '10:40',
       '14:20', '09:55', '09:30', '12:10', '15:02', '15:40', '11:40',
       '13:40', '08:20', '14:50', '08:25', '14:00', '09:00', '10:10',
       '10:30', '15:20', '08:35', '09:26', '13:30', '08:30', '10:20',
       '09:20', '13:00', '17:10', '11:42', '16:00', '13:15', '12:15',
       '11:30', '12:25', '12:35', '15:30', '14:10', '12:00', '09:45',
       '12:50', '21:10', '10:15', '10:50', '13:05', '16:30', '12:30',
       '13:59', '16:15', '12:05', '08:40', '10:35', '15:50', '08:00',
       '11:55', '10:24', '10:38', '08:45', '14:15', '12:20', '15:32',
       '11:23', '12:45', '09:35', '09:50', '11:48', '14:53', '15:05',
       '11:35', '09:40', '17:00', '11:45', '16:10', '15:10', '11:10',
       '18:30', '13:25', '13:55', '09:10', '22:55', '10:55', '14:40',
       '13:08', '11:05', '14:55', '09:09', '15:45', '15:15', '12:32',
       '11:15', '08: